# UTS Authorization and Calling - R
This notebook is an example of how to start using the UTS REST API from R

## UTS Account
If you do not have a UTS account:

* You will need to wait for an account to be activated before following this guide.
* Create a bookmark of this page, or [click here](#) to schedule a reminder.
* Follow this link to Sign-up for a [UTS License](https://uts.nlm.nih.gov/license.html).

## UTS API Key
You will need your UTS API key.  To obtain your key:

* In a new window, open the [UTS login](https://uts.nlm.nih.gov//uts.html) page.
* Once you have logged in, click on `My Profile` and copy your API key into the cell below.

In [81]:
UTS_API_KEY = 'XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX' 

You'll need to load some libraries, here these are already installed and must just be loaded:

In [129]:
library(httr)
library(jsonlite)
library(xml2)

Each UTS session includes three steps:

1. Using your API key to obtain a Ticket Granting Ticket (TGT), which is good for 8 hours.
1. Use the TGT to obtain a single use Server Ticket (ST), which is good for 1 call within the next 5 minutes.
1. Use the ST to make a single API call.

In this session, we will develop a Python class to manage this, so that you merely need to make API calls.

## Step 1 - obtain a Ticket Granting Ticket (TGT)

Let's do the first step manually, and obtain a Ticket Granting Ticket (TGT), first.

Below, we will:
* Do a POST to https://utslogin.nlm.nih.gov/cas/v1/api-key, passing our API Key as data to the call.
* Interpret the results as HTML.
* Parse the TGT from the HTML.

First, let's get the results and show the status code and content type:

In [130]:
# post to the CAS endpoint
response <- POST('https://utslogin.nlm.nih.gov/cas/v1/api-key', encode='form', body=list(apikey = apikey))

# print out the status_code and content_type
status_code(response)
headers(response)$`content-type`

[1] 201

[1] "text/html;charset=UTF-8"

R will automatically parse this as XML.   We need to do some XPath processing on the XML document to get the TGT

In [131]:
doc = content(response)
action_uri = xml_text(xml_find_first(doc, '//form/@action'))
action_uri

[1] "https://utslogin.nlm.nih.gov/cas/v1/api-key/TGT-1941074-KrQoHJQtb2HSTQYaRmaEYG0nEZmB74211HltgJXIBir2A9gCmV-cas"

Now that we have the TGT embedded in the Action URI above, let's move on to step 2:

## Step 2 - obtain a Service Ticket (ST)

We will do this by posting to the action_uri with the service URL http://umlsks.nlm.nih.gov.

In [132]:
response <- POST(action_uri, encode='form', body=list(service = 'http://umlsks.nlm.nih.gov'))
## R tries to automatically guess the body content type, let's tell it this is pure text
ticket = content(response, 'text')
ticket

[1] "ST-16047140-xBZsCK4XdxzUcgtr1QQu-cas"

## Step 3 - make an API call with the Service Ticket (ST)

In this example, we will lookup the CUI by name for diabetic foot.

* We will submit a get request to https://uts-ws.nlm.nih.gov/rest/search/current
* The `ticket` parameter will be the service ticket (ST) obtained above
* The `search` parameter will be the text we want to search for
* The `searchType` will be equal to "exact"

In [133]:
# build search_uri using the paste function for string concatenation
version = 'current'
search_uri = paste('https://uts-ws.nlm.nih.gov/rest/search/', version, sep='')

# pass the the query params into httr GET to get the response 
query_string = 'diabetic foot'
response <- GET(search_uri, query=list(ticket=ticket, string=query_string, searchType='exact'))

## print out some of the results
search_uri
status_code(response)
headers(response)$`content-type`

[1] "https://uts-ws.nlm.nih.gov/rest/search/current"

[1] 200

[1] "application/json;charset=UTF-8"

In [167]:
search_results_auto_parsed = content(response)
search_results_auto_parsed

$pageSize
[1] 25

$pageNumber
[1] 1

$result
$result$classType
[1] "searchResults"

$result$results
$result$results[[1]]
$result$results[[1]]$ui
[1] "C0206172"

$result$results[[1]]$rootSource
[1] "MTH"

$result$results[[1]]$uri
[1] "https://uts-ws.nlm.nih.gov/rest/content/2017AB/CUI/C0206172"

$result$results[[1]]$name
[1] "Diabetic Foot"

So, the JSON parsing here is pretty basic, and the class of our results is a list:

In [182]:
class(search_results_auto_parsed$result$results)

[1] "list"

For this simple result, that's fine, but suppose we are using R to call NCBI E-utilities or something.   An alternative is to use the jsonlite libraries `fromLite` function to parse it out as a dataframe:

In [171]:
search_results_data_frame = fromJSON(content(response,'text'))
search_results_data_frame
class(search_results_data_frame$result$results)

ui,rootSource,uri,name
C0206172,MTH,https://uts-ws.nlm.nih.gov/rest/content/2017AB/CUI/C0206172,Diabetic Foot


[1] "data.frame"

## Additional API calls

Whether you choose to use lists of lists or dataframes, you are going to want to call APIs on the results.  Once you've obtained the ticket granting ticket (TGT), above embeddded in our `action_uri`, you need to perform Step 2 and then 3 for each API call.  Below, we parse a URI out of the first result returnned above and retrieve its data using a new ticket.

To do this, let's make a function that gets a new ticket, and then retrieves a URI:

In [178]:
get_uts_data = function(uri, query=list()) {
    st_response = POST(action_uri, encode='form', body=list(service = 'http://umlsks.nlm.nih.gov'))
    query['ticket'] = content(st_response,'text')
    content(GET(uri, query=query))
}

Now let's apply that function to the uri of each CUI in the result above:

In [193]:
lapply(search_results_auto_parsed$result$results, function(r){ get_uts_data(r$uri) })

[[1]]
[[1]]$pageSize
[1] 25

[[1]]$pageNumber
[1] 1

[[1]]$pageCount
[1] 1

[[1]]$result
[[1]]$result$classType
[1] "Concept"

[[1]]$result$ui
[1] "C0206172"

[[1]]$result$suppressible
[1] FALSE

[[1]]$result$dateAdded
[1] "04-12-1994"

[[1]]$result$majorRevisionDate
[1] "03-08-2013"

[[1]]$result$status
[1] "R"

[[1]]$result$semanticTypes
[[1]]$result$semanticTypes[[1]]
[[1]]$result$semanticTypes[[1]]$name
[1] "Disease or Syndrome"

[[1]]$result$semanticTypes[[1]]$uri
[1] "https://uts-ws.nlm.nih.gov/rest/semantic-network/2017AB/TUI/T047"



[[1]]$result$atomCount
[1] 76

[[1]]$result$attributeCount
[1] 0

[[1]]$result$cvMemberCount
[1] 0

[[1]]$result$atoms
[1] "https://uts-ws.nlm.nih.gov/rest/content/2017AB/CUI/C0206172/atoms"

[[1]]$result$definitions
[1] "https://uts-ws.nlm.nih.gov/rest/content/2017AB/CUI/C0206172/definitions"

[[1]]$result$relations
[1] "NONE"

[[1]]$result$defaultPreferredAtom
[1] "https://uts-ws.nlm.nih.gov/rest/content/2017AB/CUI/C0206172/atoms/preferred"

[[1]]$result$relationCount
[1] 0

[[1]]$result$name
[1] "Diabetic Foot"

## Conclusion

There's a lot more we can do with this, and dress it up to be a real SDK.  We could add support for long-living code, and regenerate the TGT, that is, the action URI, if there was an error.  We can add some code to pull error messages from HTML content.   We'll leave that to you, but assure you that the authorization 